# Load Packages and Dependency
Please restart the environment after runing the following two sections.

In [ ]:
import spacy
!pip uninstall spacy

Uninstalling spacy-2.2.4:
  Would remove:
    /usr/local/bin/spacy
    /usr/local/lib/python3.7/dist-packages/bin/*
    /usr/local/lib/python3.7/dist-packages/spacy-2.2.4.dist-info/*
    /usr/local/lib/python3.7/dist-packages/spacy/*
  Would not remove (might be manually added):
    /usr/local/lib/python3.7/dist-packages/bin/theano_cache.py
Proceed (y/n)? y
  Successfully uninstalled spacy-2.2.4


In [ ]:
!pip install flair
!pip install spacy
!python -m spacy download en_core_web_trf

     |████████████████████████████████| 286kB 8.1MB/s 
     |████████████████████████████████| 19.7MB 1.2MB/s 
     |████████████████████████████████| 2.3MB 47.8MB/s 
     |████████████████████████████████| 1.2MB 67.5MB/s 
     |████████████████████████████████| 983kB 67.6MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 776.8MB 23kB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 798kB 59.5MB/s 
     |████████████████████████████████| 901kB 50.1MB/s 
     |████████████████████████████████| 3.3MB 59.0MB/s 
  Created wheel for gdown: filename=gdown-3.12.2-cp37-none-any.whl size=9705 sha256=c76d491c9bce991c4ec96a431616cab75575c52711013b48b8e116985d2d6cee
  Stored in directory: /root/.cache/pip/wheels/81/d0/d7/d9983facc6f2775411803e0e2d30ebf98efbf2fc6e57701e09
Successfully built gdown
  Created wheel for segtok: filenam

     |████████████████████████████████| 12.8MB 8.4MB/s 
     |████████████████████████████████| 9.1MB 51.2MB/s 
     |████████████████████████████████| 624kB 53.2MB/s 
     |████████████████████████████████| 460kB 57.7MB/s 
     |████████████████████████████████| 51kB 8.7MB/s 
     |████████████████████████████████| 122kB 67.5MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=a02fdb1016ba8adcefac05d4beea62b6e43cf4fde3ee635442a6af82971a65ed
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found e

2021-06-13 11:00:42.724294: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 31kB/s 
     |████████████████████████████████| 1.0MB 9.2MB/s 
     |████████████████████████████████| 2.1MB 14.6MB/s 
  Found existing installation: transformers 4.6.1
    Uninstalling transformers-4.6.1:
      Successfully uninstalled transformers-4.6.1
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [ ]:
import pandas as pd
import numpy as np
import nltk
import os
import torch
import gensim.downloader as api
import gensim
import gensim.corpora as corpora
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from flair.embeddings import TransformerWordEmbeddings, FlairEmbeddings, CharacterEmbeddings, StackedEmbeddings, WordEmbeddings
from flair.data import Sentence
import spacy
from spacy.tokens import Doc
from nltk.tag import hmm
from nltk import word_tokenize
nltk.download('brown')
from nltk.corpus import brown

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


# Data And Input

## Data Download and Load

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
if not os.path.exists('/content/train.csv'):
    link = '1_M43BgxQXXy1L9W0kcUANS8NMuO2ta38'  # Restricted shared link
    downloaded = drive.CreateFile({'id':link}) 
    downloaded.GetContentFile('train.csv')

    link = '1HsguscJAhcKUmZ1huXrcWm4VvEIe6A0k'  # Restricted shared link
    downloaded = drive.CreateFile({'id':link}) 
    downloaded.GetContentFile('val.csv')

    link = '1bUI2EhX8_Rx1Qx56oQhcWAMSMday16BR'  # Restricted shared link
    downloaded = drive.CreateFile({'id':link}) 
    downloaded.GetContentFile('test_without_labels.csv')

    link = '1I6O-6a2yQYBf3EuimuA-M0BbmN6Jv_Eg'  # Restricted shared link
    downloaded = drive.CreateFile({'id':link}) 
    downloaded.GetContentFile('sample.csv')

train = pd.read_csv('/content/train.csv')
val = pd.read_csv('/content/val.csv')
test = pd.read_csv('/content/test_without_labels.csv')
sample = pd.read_csv('/content/sample.csv')

train.shape, val.shape, test.shape

((573, 2), (191, 2), (199, 1))

## Data Preprocessing



### Converting to Lowercase
This process can group words together the inflected forms of a word like 'Everything' and 'everything', so they can be analysed as a single item.

In [ ]:
train['sents'] = train['sents'].str.lower()
val['sents'] = val['sents'].str.lower()
test['sents'] = test['sents'].str.lower()

### Intergrate Sentences

In [ ]:
whole_sents = []    # (963 items)
for s in train['sents']:
    whole_sents.append(s)
for s in val['sents']:
    whole_sents.append(s)
for s in test['sents']:
    whole_sents.append(s)

In [ ]:
max_seq_len = max(len(s.split(' ')) for s in whole_sents)
max_seq_len # 154

154

### Make Word Set & Word List

In [ ]:
word_list = []
for s in whole_sents:
    word_list.extend(s.split(' '))

word_set = set(word_list)
word_list = list(word_set)
word_list.sort()

len(word_list) # 3957

3957

### Split Data

In [ ]:
train_data, target_y_train = [s.split(' ') for s in train['sents']], [s.split(' ') for s in train['labels']]
validation_data, target_y_validation = [s.split(' ') for s in val['sents']], [s.split(' ') for s in val['labels']]
test_data = [s.split(' ') for s in test['sents']]

len(train_data), len(validation_data), len(test_data), 

(573, 191, 199)

### Generate word_to_ix, tag_to_ix, ix_to_tag

In [ ]:
word_to_ix = {}
for sentence in train_data+validation_data+test_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train+target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)
ix_to_tag={}
for tag in tag_to_ix:
    ix_to_tag[tag_to_ix[tag]] = tag

### Convert Dataset into Idxs

In [ ]:
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(train_data,word_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)
len(train_input_index[0])

52

## Input Embedding

### Domain Embedding

#### Download and process domain dataset

In [ ]:
# download the whole re3d dataset
%cd /content
!git clone https://github.com/juand-r/entity-recognition-datasets.git
train_re3d = open("/content/entity-recognition-datasets/data/re3d/CONLL-format/data/train/re3d-train.conll", "r")
test_re3d = open("/content/entity-recognition-datasets/data/re3d/CONLL-format/data/test/re3d-test.conll", "r")
train_re3d = train_re3d.read().splitlines()
test_re3d = test_re3d.read().splitlines()

for i,v in enumerate(train_re3d):
    train_re3d[i] = v.split('\t')[0]
for i,v in enumerate(test_re3d):
    test_re3d[i] = v.split('\t')[0]

train_re3d = train_re3d[1:]
test_re3d = test_re3d[1:]

/content
Cloning into 'entity-recognition-datasets'...
remote: Enumerating objects: 1050, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1050 (delta 6), reused 0 (delta 0), pack-reused 1038
Receiving objects: 100% (1050/1050), 2.43 MiB | 23.92 MiB/s, done.
Resolving deltas: 100% (425/425), done.


In [ ]:
train_sents = []
sent=''
for i in train_re3d:
    if i == '':
        sent = sent[:-1]
        train_sents.append(sent)
        sent = ''
    else:
        sent += i
        sent += ' '


test_sents = []
sent=''
for i in test_re3d:
    if i == '':
        sent = sent[:-1]
        test_sents.append(sent)
        sent = ''
    else:
        sent += i
        sent += ' '

whole_sents = train_sents + test_sents
len(whole_sents), len(train_sents), len(test_sents)

(963, 764, 199)

In [ ]:
if not os.path.exists("/content/corpus/train"):
    os.makedirs("/content/corpus/train")
f = open("/content/corpus/train/train_split_1.txt","w")
f.writelines("%s\n" % l for l in whole_sents)
f.close()

f = open("/content/corpus/test.txt","w")
f.writelines("%s\n" % l for l in test_sents)
f.close()

f = open("/content/corpus/valid.txt","w")
f.writelines("%s\n" % l for l in test_sents)
f.close()

#### Generate Domain Embeddings
We train a Domain Embeddings object by `FlairEmbedding`

In [ ]:
%cd /content
from flair.data import Dictionary
from flair.models import LanguageModel
from flair.trainers.language_model_trainer import LanguageModelTrainer, TextCorpus

# are you training a forward or backward LM?
is_forward_lm = True

# load the default character dictionary
dictionary: Dictionary = Dictionary.load('chars')

# get your corpus, process forward and at the character level
corpus = TextCorpus('/content/corpus',
                    dictionary,
                    is_forward_lm,
                    character_level=True)

# instantiate your language model, set hidden size and number of layers
language_model = LanguageModel(dictionary,
                               is_forward_lm,
                               hidden_size=128,
                               nlayers=1)

# train your language model
trainer = LanguageModelTrainer(language_model, corpus)

trainer.train('resources/taggers/language_model',
              sequence_length=10,
              mini_batch_size=10,
              max_epochs=10)


/content
2021-06-13 11:04:10,220 https://flair.informatik.hu-berlin.de/resources/characters/common_characters not found in cache, downloading to /tmp/tmp149klhe_


100%|██████████| 2887/2887 [00:00<00:00, 1026008.78B/s]

2021-06-13 11:04:10,700 copying /tmp/tmp149klhe_ to cache at /root/.flair/datasets/common_characters
2021-06-13 11:04:10,703 removing temp file /tmp/tmp149klhe_
2021-06-13 11:04:10,705 read text file with 199 lines
2021-06-13 11:04:10,877 read text file with 199 lines


2021-06-13 11:04:13,273 read text file with 963 lines
2021-06-13 11:04:13,277 shuffled
2021-06-13 11:04:14,089 Sequence length is 10
2021-06-13 11:04:14,094 Split 1	 - (11:04:14)
2021-06-13 11:04:14,392 | split   1 /  1 |   100/ 1351 batches | ms/batch  2.95 | loss  3.37 | ppl    28.97
2021-06-13 11:04:14,680 | split   1 /  1 |   200/ 1351 batches | ms/batch  2.86 | loss  2.73 | ppl    15.35
2021-06-13 11:04:14,967 | split   1 /  1 |   300/ 1351 batches | ms/batch  2.86 | loss  2.53 | ppl    12.58
2021-06-13 11:04:15,246 | split   1 /  1 |   400/ 1351 batches | ms/batch  2.78 | loss  2.41 | ppl    11.16
2021-06-13 11:04:15,529 | split   1 /  1 |   500/ 1351 batches | ms/batch  2.81 | loss  2.39 | ppl    10.96
2021-06-13 11:04:15,817 | split   1 /  1 |   600/ 1351 batches | ms/batch  2.87 | loss  2.39 | ppl    10.86
2021-06-13 11:04:16,104 | split   1 /  1 |   700/ 1351 batches | ms/batch  2.86 | loss  2.28 | ppl     9.74
2021-06-13 11:04:16,388 | split   1 /  1 |   800/ 1351 batches | 

### Syntactic Embedding



####Embedding Functions

##### TF-IDF

In [ ]:
#refer: https://www.tutorialspoint.com/gensim/gensim_creating_tf_idf_matrix.htm
whole_token = [sentence.split(' ') for sentence in whole_sents] # (963 items) [['Operation', 'Steel', 'Curtain', '(', 'Arabic', ...], ['The', ....
whole_dic = corpora.Dictionary()    # Dictionary with 4291 items--> 3957
whole_bow = [whole_dic.doc2bow(words, allow_update=True) for words in whole_token]  # (963 items) [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), ...], [(2, 1), (14, 1), ......
whole_tfidf = gensim.models.TfidfModel(whole_bow, smartirs='ntc')

def tfidf_f(train,val,test):
    def get_tfidf(sentences):
        sentence_list = [sent for sent in sentences['sents']]
        sentence_token = [row.split(' ') for row in sentence_list]
        dic = corpora.Dictionary()
        bow = [dic.doc2bow(words, allow_update=True) for words in sentence_token]
        freq_list = []
        for doc in whole_tfidf[bow]:
            freq_dic = {}
            for id, freq in doc:
                freq_dic[whole_dic[id]] = round(freq, 2)
            freq_list.append(freq_dic)
        sentences_freq = []    #(573 items) [[[0.17], [0.14], [0.14], [0.09], [0.12], ...], [[0.02], [0.2.....
        for i,words in enumerate(sentence_token): # sentences
            temp = []
            for j, word in enumerate(words): # word
                if word in freq_list[i]:
                    # print('true')
                    token_freq = freq_list[i][word]
                    temp.append([token_freq])
                else:
                    token_freq = 0
                    temp.append([token_freq])
            sentences_freq.append(temp)
        return sentences_freq

    train_tfidf = get_tfidf(train)  # (573 items) [[0.08, 0.08, 0.0, 0.15, 0.07, ...], [0.0, 0.18, 0.5, ......
    val_tfidf = get_tfidf(val)  # (191 items) [[0.52, 0.1, 0.0, 0.19, 0.09, ...], [0.13, 0.0, 0.......
    test_tfidf = get_tfidf(test)    # (199 items) [[0.11, 0.22, 0.0, 0.2, 0.09, ...], [0.48, 0.22, 0.1,......   
    tfidf_dim = 1

    return train_tfidf, val_tfidf, test_tfidf, tfidf_dim
# tr_idf, vv_idf, tt_idf, idf_dd = tfidf_f(train, val, test)

##### PoS tag

In [ ]:
# for PoS tagging
def pos_f(train, val, test, type='spacy', feature = 'tag'):
    assert type in {'spacy', 'hmm'} and feature in {'tag', 'pos_tag'}, "wrong type! "
    if type == 'hmm' and feature == 'tag':
        brown_tagged_sents = brown.tagged_sents()
        trainer = hmm.HiddenMarkovModelTrainer() 
        trained_tagger = trainer.train_supervised(brown_tagged_sents) 

        train_sentence_list = [sentence for sentence in train['sents']]
        train_sentence_token = [row.split(' ') for row in train_sentence_list]
        train_tag_temp = [trained_tagger.best_path(sentence) for sentence in train_sentence_token]

        val_sentence_list = [sentence for sentence in val['sents']]
        val_sentence_token = [row.split(' ') for row in val_sentence_list]
        val_tag_temp = [trained_tagger.best_path(sentence) for sentence in val_sentence_token]

        test_sentence_list = [sentence for sentence in test['sents']]
        test_sentence_token = [row.split(' ') for row in test_sentence_list]
        test_tag_temp = [trained_tagger.best_path(sentence) for sentence in test_sentence_token]  

        whole_hmm = train_tag_temp + val_tag_temp + test_tag_temp

        all_hmm = set()
        for sentence in whole_hmm:
            for word_hmm in sentence:
                all_hmm.add(word_hmm)

        hmm_list = list(all_hmm)
        pos_dim = len(hmm_list)

        train_hmm_vec = tag_to_OHvector(train_tag_temp, hmm_list) # 573 (573 items) [[[0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, ...], [0, 0, 0, 
        val_hmm_vec = tag_to_OHvector(val_tag_temp, hmm_list)     # 191
        test_hmm_vec = tag_to_OHvector(test_tag_temp, hmm_list)   # 199

        return train_hmm_vec, val_hmm_vec, test_hmm_vec, pos_dim

    elif type == 'spacy' and feature == 'pos_tag':
        def pos_tagging(sentences):
            pos = [] # train_pos
            for sentence in sentences:
                tokens = nlp(sentence)
                sub_list = []
                for token in tokens:
                    sub_list.append(token.pos_) # .pos_ --> readable tag // .pos --> hash value
                pos.append(sub_list)
            return pos
        train_pos = pos_tagging(train['sents']) # (573 items) [['PROPN', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', ...], ['DET', 'NOUN',......
        val_pos = pos_tagging(val['sents'])     # 191
        test_pos = pos_tagging(test['sents'])   # 199

        whole_pos = train_pos + val_pos + test_pos  # 963

        all_pos = set()
        for sentence in whole_pos:
            for word_pos in sentence:
                all_pos.add(word_pos)

        pos_list = list(all_pos)
        pos_dim = len(pos_list)

        train_pos = tag_to_OHvector(train_pos, pos_list) # 573 (573 items) [[[0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, ...], [0, 0, 0, 
        val_pos = tag_to_OHvector(val_pos, pos_list)     # 191
        test_pos = tag_to_OHvector(test_pos, pos_list)   # 199

        return train_pos, val_pos, test_pos, pos_dim

    elif type == 'spacy' and feature == 'tag':
        def pos_tagging(sentences):
            pos = [] # train_pos
            for sentence in sentences:
                tokens = nlp(sentence)
                sub_list = []
                for token in tokens:
                    sub_list.append(token.tag_) # .tag_ --> readable tag // .tag --> hash value
                pos.append(sub_list)
            return pos
        train_pos = pos_tagging(train['sents']) # (573 items) [['PROPN', 'NOUN', 'NOUN', 'PUNCT', 'PROPN', ...], ['DET', 'NOUN',......
        val_pos = pos_tagging(val['sents'])     # 191
        test_pos = pos_tagging(test['sents'])   # 199

        whole_pos = train_pos + val_pos + test_pos  # 963

        all_pos = set()
        for sentence in whole_pos:
            for word_pos in sentence:
                all_pos.add(word_pos)

        pos_list = list(all_pos)
        pos_dim = len(pos_list)

        train_pos = tag_to_OHvector(train_pos, pos_list) # 573 (573 items) [[[0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, ...], [0, 0, 0, 
        val_pos = tag_to_OHvector(val_pos, pos_list)     # 191
        test_pos = tag_to_OHvector(test_pos, pos_list)   # 199

        return train_pos, val_pos, test_pos, pos_dim

# tr_pos, vv_pos, tt_pos, dim_pos = pos_f(train, val, test,'hmm','tag')

##### Dependency Path

In [ ]:
# for Dependency Parsing
def dep_f(train, val, test):
    def dependency(sentences):
        pos = [] # train_pos
        for sentence in sentences:
            words = nlp(sentence)
            sub_list = []
            for token in words:
                sub_list.append(token.dep_)
            pos.append(sub_list)
        return pos
    train_dep = dependency(train['sents']) # 573
    val_dep = dependency(val['sents'])     # 191
    test_dep = dependency(test['sents'])   # 199

    whole_dep = train_dep + val_dep + test_dep  # 963

    all_dep = set()
    for sentence in whole_dep:
        for word_pos in sentence:
            all_dep.add(word_pos)

    dep_list = list(all_dep)
    dep_dim = len(dep_list) # 44

    train_dep = tag_to_OHvector(train_dep, dep_list) # (573 items) [[[0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, ...], [0, 0,......
    val_dep = tag_to_OHvector(val_dep, dep_list)
    test_dep = tag_to_OHvector(test_dep, dep_list)

    return train_dep, val_dep, test_dep, dep_dim

# tr_dep, vv_dep, tt_dep, dd = dep_f(train, val, test)

##### Word shape

In [ ]:
# for word_shape
def word_shape_f(train, val, test):
    def word_shape(sentences):
        pos = [] # train_pos
        for sentence in sentences:
            words = nlp(sentence)
            sub_list = []
            for token in words:
                sub_list.append(token.shape_)
            pos.append(sub_list)
        return pos
    train_len = word_shape(train['sents']) # 573
    val_len = word_shape(val['sents'])     # 191
    test_len = word_shape(test['sents'])   # 199

    whole_len = train_len + val_len + test_len  # 963

    all_len = set()
    for sentence in whole_len:
        for word_len in sentence:
            all_len.add(word_len)

    len_list = list(all_len)
    len_dim = len(len_list) # 44

    train_len = tag_to_OHvector(train_len, len_list) # (573 items) [[[0, 0, 0, 0, 0, ...], [0, 0, 0, 0, 0, ...], [0, 0,......
    val_len = tag_to_OHvector(val_len, len_list)
    test_len = tag_to_OHvector(test_len, len_list)

    return train_len, val_len, test_len, len_dim

# tr_len, vv_len, tt_len, dd_len = word_shape_f(train, val, test)

#### Util Functions

In [ ]:
class WhitespaceTokenizer:
    def __init__(self, vocab):
        self.vocab = vocab

    def __call__(self, text):
        words = text.split(" ")
        return Doc(self.vocab, words=words)

nlp = spacy.load("en_core_web_trf")
nlp.tokenizer = WhitespaceTokenizer(nlp.vocab)

In [ ]:
# convert tags into OneHot vector
def tag_to_OHvector(pos, unique_list):
    pos_vec = []
    for sentence in pos:
        sent_temp = [] 
        for word in sentence:
            word_temp = [0] * len(unique_list) # 17
            if word in unique_list:
                word_temp[unique_list.index(word)] = 1
            sent_temp.append(word_temp)
        pos_vec.append(sent_temp)
    return pos_vec

In [ ]:
# for getting optional syntactic features
def syn_vectors_f(train, val, test, type='spacy', feature = 'tag'):
    assert type in {'spacy', 'hmm','gensim'} and feature in {'tag', 'pos_tag', 'dep', 'word_shape','tfidf'}, 'wrong type! should in tag, pos_tag, dep, word_shape'
    if type == 'hmm' and feature == 'tag':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = pos_f(train, val, test, 'hmm', 'tag')
    elif type == 'spacy' and feature == 'tag':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = pos_f(train, val, test, 'spacy', 'tag')
    elif type == 'spacy' and feature == 'pos_tag':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = pos_f(train, val, test, 'spacy', 'pos_tag')
    elif type == 'spacy' and feature == 'dep':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = dep_f(train, val, test)
    elif type == 'spacy' and feature == 'word_shape':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = word_shape_f(train, val, test)
    elif type == 'gensim' and feature == 'tfidf':
        train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = tfidf_f(train, val, test)
    
    return train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM

In [ ]:
# for getting converted vectors
def vector_concat(train, val, test, vec_type=['spacy', 'hmm'], feature = ['tag','tag']):
    assert len(vec_type) == len(feature), 'Wrong Input! '
    if len(vec_type)==1:
        train_torch_all, val_torch_all, test_torch_all, vec_dim = syn_vectors_f(train, val, test, vec_type[0], feature[0])        
    else:
        train_dic = {} #{'0':...,'1':..., '2':...,'3':...,'4':....}
        val_dic = {}
        test_dic = {}
        dim_dic = {}
        i = 0
        for tup in zip(vec_type, feature):
            print("Training: ", i, tup)
            train_dic[i], val_dic[i], test_dic[i], dim_dic[i] = syn_vectors_f(train, val, test, tup[0],tup[1])
            i += 1
        vec_dim = sum(dim_dic.values())
        print(len(train_dic), len(val_dic), len(test_dic), vec_dim) # 3 3 3 139

        # use train_dic !
        train_torch = {}     # {0: [], 1: [], 2: []}
        for i in range(len(vec_type)):
            print(i)
            train_torch[i] = []
            ss = 0
            for sent in train_dic[i]: # train_dic[0]
                train_dic[i][ss] = torch.tensor(train_dic[i][ss], dtype=torch.long).to(device) 
                aaa = train_dic[i][ss].view(train_dic[i][ss].shape[0], 1, -1)
                ss += 1
                train_torch[i].append(aaa)
        train_torch_all = []
        for i in range(len(train_torch[0])):
            if len(vec_type)==2:
                temp = torch.cat((train_torch[0][i], train_torch[1][i]), 2) ########
                train_torch_all.append(temp)
            elif len(vec_type)==3:
                temp = torch.cat((train_torch[0][i], train_torch[1][i], train_torch[2][i]), 2)
                train_torch_all.append(temp)
            elif len(vec_type)==4:
                temp = torch.cat((train_torch[0][i], train_torch[1][i], train_torch[2][i], train_torch[3][i]), 2)
                train_torch_all.append(temp)
            elif len(vec_type)==5:
                temp = torch.cat((train_torch[0][i], train_torch[1][i], train_torch[2][i], train_torch[3][i], train_torch[4][i]), 2)
                train_torch_all.append(temp)
            elif len(vec_type)==6:
                temp = torch.cat((train_torch[0][i], train_torch[1][i], train_torch[2][i], train_torch[3][i], train_torch[4][i],train_torch[5][i]),  2)
                train_torch_all.append(temp)

        # use val_dic !
        val_torch = {}     # {0: [], 1: [], 2: []}
        for i in range(len(vec_type)):
            print(i)
            val_torch[i] = []
            ss = 0
            for sent in val_dic[i]: # val_dic[0]
                val_dic[i][ss] = torch.tensor(val_dic[i][ss], dtype=torch.long).to(device) 
                aaa = val_dic[i][ss].view(val_dic[i][ss].shape[0], 1, -1)
                ss += 1
                val_torch[i].append(aaa)
        val_torch_all = []
        for i in range(len(val_torch[0])):
            if len(vec_type)==2:
                temp = torch.cat((val_torch[0][i], val_torch[1][i]), 2) 
                val_torch_all.append(temp)
            elif len(vec_type)==3:
                temp = torch.cat((val_torch[0][i], val_torch[1][i], val_torch[2][i]), 2)
                val_torch_all.append(temp)
            elif len(vec_type)==4:
                temp = torch.cat((val_torch[0][i], val_torch[1][i], val_torch[2][i], val_torch[3][i]), 2)
                val_torch_all.append(temp)
            elif len(vec_type)==5:
                temp = torch.cat((val_torch[0][i], val_torch[1][i], val_torch[2][i], val_torch[3][i], val_torch[4][i]), 2)
                val_torch_all.append(temp)
            elif len(vec_type)==6:
                temp = torch.cat((val_torch[0][i], val_torch[1][i], val_torch[2][i], val_torch[3][i], val_torch[4][i],val_torch[5][i]), 2)
                val_torch_all.append(temp)

        # use train_dic !
        test_torch = {}     # {0: [], 1: [], 2: []}
        for i in range(len(vec_type)):
            print(i)
            test_torch[i] = []
            ss = 0
            for sent in test_dic[i]: # test_dic[0]
                test_dic[i][ss] = torch.tensor(test_dic[i][ss], dtype=torch.long).to(device) 
                aaa = test_dic[i][ss].view(test_dic[i][ss].shape[0], 1, -1)
                ss += 1
                test_torch[i].append(aaa)
        test_torch_all = []
        for i in range(len(test_torch[0])):
            if len(vec_type)==2:
                temp = torch.cat((test_torch[0][i], test_torch[1][i]), 2)
                test_torch_all.append(temp)
            elif len(vec_type)==3:
                temp = torch.cat((test_torch[0][i], test_torch[1][i], test_torch[2][i]), 2)
                test_torch_all.append(temp)
            elif len(vec_type)==4:
                temp = torch.cat((test_torch[0][i], test_torch[1][i], test_torch[2][i], test_torch[3][i]), 2)
                test_torch_all.append(temp)
            elif len(vec_type)==5:
                temp = torch.cat((test_torch[0][i], test_torch[1][i], test_torch[2][i], test_torch[3][i], test_torch[4][i]), 2)
                test_torch_all.append(temp)
            elif len(vec_type)==6:
                temp = torch.cat((test_torch[0][i], test_torch[1][i], test_torch[2][i], test_torch[3][i], test_torch[4][i], test_torch[5][i]), 2)
                test_torch_all.append(temp)

    # return train_dic, val_dic, test_dic, dim_dic, vec_dim
    return train_torch_all, val_torch_all, test_torch_all, vec_dim

#### Generate Syntactic Embeddings

In [ ]:
train_syn_vectors, val_syn_vectors, test_syn_vectors, SYN_VECTOR_DIM = vector_concat(train, 
                                                                                     val, 
                                                                                     test, 
                                                                                     ['gensim','spacy','spacy'],
                                                                                     ['tfidf','tag','pos_tag'])
SYN_VECTOR_DIM

Training:  0 ('gensim', 'tfidf')
Training:  1 ('spacy', 'tag')
Training:  2 ('spacy', 'pos_tag')
3 3 3 63
0
1
2
0
1
2
0
1
2


63

### Semantic Embedding


#### Word Embeddings Function

In [ ]:
def word_emb_f(corpus="glove-wiki-gigaword-100"):
    word_emb_model = api.load(corpus)
    return word_emb_model, word_emb_model.vector_size

#### Generate Word Embedding Matrix

In [ ]:
import gensim.downloader as api
word_emb_model, WORD_EMBEDDING_DIM = word_emb_f(corpus="glove-wiki-gigaword-100")

embedding_matrix = []
for word in word_list:
    try:
        embedding_matrix.append(word_emb_model.wv[word])
    except:
        embedding_matrix.append([0]*WORD_EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape # (3957, 100)

[==================================================] 100.0% 128.1/128.1MB downloaded


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  import sys


(3957, 100)

#### Contextual Embeddings

In [ ]:
from flair.embeddings import TransformerWordEmbeddings, FlairEmbeddings, CharacterEmbeddings, StackedEmbeddings, WordEmbeddings

#NER Model

###BiLSTM+Attention+CRF


In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F


torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, word_embedding_dim, syntactic_feature_dim, hidden_dim, attention_method=None, use_CRF=False):
        super(BiLSTM_CRF, self).__init__()
        self.word_embedding_dim = word_embedding_dim    # word_emb_dim = WORD_EMBEDDING_DIM = 100 --> giga word 100
        self.hidden_dim = hidden_dim # hidden_state_dim = 50 --> hidden_size --> n
        self.vocab_size = vocab_size # len(word_to_ix) --> 3957 (3957 items) {'operation': 0, 'steel': 1, 'curtain': 2, '(': 3, 'arabic': 
        self.tag_to_ix = tag_to_ix # 23 {'<START>': 0, '<STOP>': 1, 'O': 2, 'B-Organisation': 3,....
        self.tagset_size = len(tag_to_ix) # 23
        self.attention_method=attention_method
        self.use_CRF = use_CRF
        self.dropout = nn.Dropout(0.5)

        if attention_method == 'dot_product':
            self.attention_layer = DotProductAttention()
        elif attention_method == 'scaled_dot_product':
            self.attention_layer = ScaledDotProductAttention()
        elif attention_method == 'cos':
            self.attention_layer = CosineAttention()
        elif attention_method == 'general':
            self.attention_layer = GeneralAttention()
        elif attention_method == 'muiti_head':
            self.attention_layer = nn.MultiheadAttention(embed_dim=hidden_dim, num_heads=5)
        elif attention_method == 'transformer':
            self.attention_layer = nn.Transformer(hidden_dim, num_encoder_layers=1).encoder # default number of head --> 8
        elif attention_method == None:
            self.attention_layer = None
        
        if attention_method != None:
            self.linear_q = nn.Linear(self.hidden_dim, self.hidden_dim)
            self.linear_k = nn.Linear(self.hidden_dim, self.hidden_dim)
            self.linear_v = nn.Linear(self.hidden_dim, self.hidden_dim)

        # self.word_embeds = nn.Embedding(vocab_size, word_embedding_dim) # 3957,100

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        # self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))  #  embedding_matrix --> ndarray with shape (3957, 100)

        self.word_embeds = StackedEmbeddings([
                                                TransformerWordEmbeddings('bert-base-uncased', fine_tune=False, layers='-1'),
                                                CharacterEmbeddings(),
                                                FlairEmbeddings('resources/taggers/language_model/best-lm.pt')
                                            ])
        

        self.lstm = nn.LSTM(word_embedding_dim + syntactic_feature_dim, 
                            hidden_dim // 2, # --> hidden_dem * 2 ?
                            num_layers=1, #1, 
                            bidirectional=True) #,batch_first=True)

        # Maps the output of the LSTM into tag space.
        self.classifier = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of transitioning *to* i *from* j.
        self.transitions = nn.Parameter(torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device), # torch.randn(2, 1, 25)
                torch.randn(2, 1, self.hidden_dim // 2).to(device)) # torch.randn(2, 1, 25)

    def _forward_alg(self, feats): # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats: # each sentence
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence, syntactic_feature):
        self.hidden = self.init_hidden()
        # embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)

        sentence = Sentence(" ".join(sentence), use_tokenizer=False)
        embeds = self.word_embeds.embed(sentence)
        embeds = torch.stack([i.embedding for i in sentence])
        embeds = embeds.view(len(sentence), 1, -1)

        if syntactic_feature != None:
            syntactic_feature = syntactic_feature.view(syntactic_feature.shape[0], 1, -1)
            embeds = torch.cat((embeds, syntactic_feature), 2)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.permute(1,0,2) 
        # (seq_len, batch, num_directions * hidden_size) --> 
        #(batch, seq_len, num_directions * hidden_size)


        if self.attention_method == 'transformer':
            lstm_out = lstm_out.permute(1,0,2)
            output = self.attention_layer(lstm_out)
        elif self.attention_layer != None:
            # Linear projection
            q = self.linear_q(lstm_out)
            k = self.linear_k(lstm_out)
            v = self.linear_v(lstm_out)
            output, attention = self.attention_layer(q,k,v)
        else:
            output = lstm_out
        
        output = output.view(len(sentence), self.hidden_dim)

        output = self.dropout(output)
        
        lstm_feats = self.classifier(output)

        return lstm_feats

    def _score_sentence(self, feats, tags):  # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):  # to Find the best path, given the features.
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags, syntactic_feature):
        feats = self._get_lstm_features(sentence, syntactic_feature)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence, syntactic_feature):  # dont confuse this with _forward_alg above.

        if syntactic_feature != None:
            syntactic_feature = torch.tensor(syntactic_feature, dtype=torch.long).to(device)
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence, syntactic_feature)

        if self.use_CRF:
            # Find the best path, given the features.
            score, tag_seq = self._viterbi_decode(lstm_feats)
        else:
            tag_seq = torch.max(F.softmax(lstm_feats, dim=1), dim=1).indices.tolist()
            score = None
        return score, tag_seq

### Attention Classes

#### DotProductAttention

In [ ]:
# refer: https://github.com/ROBINADC/BiGRU-CRF-with-Attention-for-NER
class DotProductAttention(nn.Module):
    def __init__(self, dropout_rate=0.0, **kwargs):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)
    def forward(self, q, k, v):
        attention = torch.bmm(q, k.permute(0, 2, 1))
        if attn_mask is not None:
            attention.masked_fill_(attn_mask, -np.inf)  # positions that require masking are now -np.inf
        # get attention score
        attention = F.softmax(attention, dim=-1)
        attention = self.dropout(attention)
        output = attention.bmm(v)
        return output, attention

####CosineAttention

In [ ]:
# refer: https://github.com/ROBINADC/BiGRU-CRF-with-Attention-for-NER
class CosineAttention(nn.Module):
    def __init__(self, dropout_rate=0.0, eps=1e-10, **kwargs):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)
        self.eps = eps

    def forward(self, q, k, v, attn_mask=None):
        q_norm = q / (q.norm(p=2, dim=-1, keepdim=True) + self.eps)  # (B, T_q, D)
        k_norm = k / (k.norm(p=2, dim=-1, keepdim=True) + self.eps)  # (B, T_k, D)
        attention = torch.bmm(q_norm, k_norm.permute(0, 2, 1))  # (B, T_q, T_k)
        if attn_mask is not None:
            attention.masked_fill_(attn_mask, -np.inf)  # positions that require masking are now -np.inf
        attention = F.softmax(attention, dim=-1)
        attention = self.dropout(attention)
        output = attention.bmm(v)  # (B, T_q, D_v)
        return output, attention

####ScaledDotProductAttention

In [ ]:
# refer: https://github.com/ROBINADC/BiGRU-CRF-with-Attention-for-NER
class ScaledDotProductAttention(nn.Module):
    def __init__(self, dropout_rate=0.0, **kwargs):
        super().__init__()
        self.dropout = nn.Dropout(dropout_rate)

    def forward(self, q, k, v, attn_mask=None):
        attention = torch.bmm(q, k.permute(0, 2, 1))  # (B, T_q, T_k)
        # Scale
        attention *= k.size(-1) ** -0.5
        if attn_mask is not None:
            attention.masked_fill_(attn_mask, -np.inf)  # positions that require masking are now -np.inf
        attention = F.softmax(attention, dim=-1)
        attention = self.dropout(attention)
        output = attention.bmm(v)  # (B, T_q, D_v)
        return output, attention

#### Multi-Head Self Attention

In [ ]:
import torch.nn as nn
nn.Transformer().encoder

TransformerEncoder(
  (layers): ModuleList(
    (0): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
      )
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=512, bias=True)
      (norm1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (norm2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
      (dropout1): Dropout(p=0.1, inplace=False)
      (dropout2): Dropout(p=0.1, inplace=False)
    )
    (1): TransformerEncoderLayer(
      (self_attn): MultiheadAttention(
        (out_proj): _LinearWithBias(in_features=512, out_features=512, bias=True)
      )
      (linear1): Linear(in_features=512, out_features=2048, bias=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (linear2): Linear(in_features=2048, out_features=512, bias=True)
      (norm1

### Evaluation Function

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

def cal_acc(model, input_sents, output_index, pos):
    ground_truth = []
    predicted = []
    for i,sent in enumerate(input_sents):
        ground_truth += output_index[i]
        # score, pred = model(torch.tensor(sent, dtype=torch.long).to(device), pos[i])
        if pos != None:
            _, pred = model(sent, pos[i])
        else:
            _, pred = model(sent, None)
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    f1 = f1_score(ground_truth, predicted, average='micro')
    return predicted, ground_truth, accuracy, f1.mean()

### Initialize Model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
word_emb_dim = 946
use_CRF=False

'''
if do not use syntatic embedding, 
set use_syntatic_embedding to false
'''
use_syntatic_embedding = False
if use_syntatic_embedding != True:
    train_syn_vectors = None
    val_syn_vectors = None
    test_syn_vectors = None
    SYN_VECTOR_DIM = 0


syn_vect_dim = SYN_VECTOR_DIM
hidden_state_dim = 952
epochs = 100

model = BiLSTM_CRF(vocab_size=len(word_to_ix), 
                   tag_to_ix=tag_to_ix, 
                   word_embedding_dim=word_emb_dim, 
                   syntactic_feature_dim=syn_vect_dim,
                   hidden_dim=hidden_state_dim,
                   attention_method='transformer',
                   # choose from ['dot_product', 'scaled_dot_product', 'cos', 'general', 'transformer']
                   use_CRF=True)
model.to(device)
optimizer = optim.AdamW(model.parameters(), lr=0.0001, weight_decay=1e-4)

### Train the model

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
val_loss_arr = []
val_f1_arr = []

for epoch in range(epochs):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, sent in enumerate(train_data):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        optimizer.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,µˆ
        # turn them into Tensors of word indices.
        if train_syn_vectors != None:
            train_syn_vector = torch.tensor(train_syn_vectors[i], dtype=torch.long).to(device)
        else:
            train_syn_vector = None
        # sentence_in = torch.tensor(sent, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sent, targets, train_syn_vector)
        # print("step loss:", loss)

        # Step 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc, train_f1 = cal_acc(model,train_data,train_output_index, train_syn_vectors)
    _, _, val_acc, val_f1 = cal_acc(model,validation_data,val_output_index, val_syn_vectors)

    val_loss = 0
    for i, sent in enumerate(validation_data):
        tags_index = val_output_index[i]
        # sentence_in = torch.tensor(sent, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        if val_syn_vectors != None:
            val_syn_vector = torch.tensor(val_syn_vectors[i], dtype=torch.long).to(device)
        else:
            val_syn_vector = None

        loss = model.neg_log_likelihood(sent, targets, val_syn_vector) 
        val_loss+=loss.item()
        
    time2 = datetime.datetime.now()

    if len(val_f1_arr)==0 or val_f1 > val_f1_arr[-1]:
        torch.save(model.state_dict(),'/content/best_model.pth')
        
    val_f1_arr.append(val_f1)
    print("Epoch:%d, Training Loss: %.2f, Training f1: %.2f, train acc: %.4f, val f1: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss, train_f1,train_acc, val_f1, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training Loss: 15583.01, Training f1: 0.82, train acc: 0.8217, val f1: 0.76, val acc: 0.7637, time: 256.01s
Epoch:2, Training Loss: 8494.83, Training f1: 0.89, train acc: 0.8890, val f1: 0.81, val acc: 0.8125, time: 258.83s
Epoch:3, Training Loss: 6183.07, Training f1: 0.91, train acc: 0.9099, val f1: 0.82, val acc: 0.8229, time: 259.73s
Epoch:4, Training Loss: 4912.96, Training f1: 0.92, train acc: 0.9205, val f1: 0.82, val acc: 0.8248, time: 261.62s
Epoch:5, Training Loss: 3981.74, Training f1: 0.93, train acc: 0.9290, val f1: 0.83, val acc: 0.8320, time: 260.45s
Epoch:6, Training Loss: 3459.00, Training f1: 0.94, train acc: 0.9383, val f1: 0.83, val acc: 0.8273, time: 260.57s
Epoch:7, Training Loss: 2919.77, Training f1: 0.94, train acc: 0.9437, val f1: 0.84, val acc: 0.8373, time: 264.64s
Epoch:8, Training Loss: 2766.90, Training f1: 0.95, train acc: 0.9543, val f1: 0.83, val acc: 0.8322, time: 264.74s
Epoch:9, Training Loss: 2345.42, Training f1: 0.94, train acc: 0.9448, 

KeyboardInterrupt: ignored

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime
val_loss_arr = []
val_f1_arr = []

for epoch in range(epochs):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, sent in enumerate(train_data):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        optimizer.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,µˆ
        # turn them into Tensors of word indices.
        if train_syn_vectors != None:
            train_syn_vector = torch.tensor(train_syn_vectors[i], dtype=torch.long).to(device)
        else:
            train_syn_vector = None
        # sentence_in = torch.tensor(sent, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sent, targets, train_syn_vector)
        # print("step loss:", loss)

        # Step 4. Compute the loss, gradients, and update the parameters by calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc, train_f1 = cal_acc(model,train_data,train_output_index, train_syn_vectors)
    _, _, val_acc, val_f1 = cal_acc(model,validation_data,val_output_index, val_syn_vectors)

    val_loss = 0
    for i, sent in enumerate(validation_data):
        tags_index = val_output_index[i]
        # sentence_in = torch.tensor(sent, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        if val_syn_vectors != None:
            val_syn_vector = torch.tensor(val_syn_vectors[i], dtype=torch.long).to(device)
        else:
            val_syn_vector = None

        loss = model.neg_log_likelihood(sent, targets, val_syn_vector) 
        val_loss+=loss.item()
        
    time2 = datetime.datetime.now()

    if len(val_f1_arr)==0 or val_f1 > val_f1_arr[-1]:
        torch.save(model.state_dict(),'/content/best_model.pth')
        
    val_f1_arr.append(val_f1)
    print("Epoch:%d, Training Loss: %.2f, Training f1: %.2f, train acc: %.4f, val f1: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss, train_f1,train_acc, val_f1, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training Loss: 15583.01, Training f1: 0.82, train acc: 0.8217, val f1: 0.76, val acc: 0.7637, time: 256.01s
Epoch:2, Training Loss: 8494.83, Training f1: 0.89, train acc: 0.8890, val f1: 0.81, val acc: 0.8125, time: 258.83s
Epoch:3, Training Loss: 6183.07, Training f1: 0.91, train acc: 0.9099, val f1: 0.82, val acc: 0.8229, time: 259.73s
Epoch:4, Training Loss: 4912.96, Training f1: 0.92, train acc: 0.9205, val f1: 0.82, val acc: 0.8248, time: 261.62s
Epoch:5, Training Loss: 3981.74, Training f1: 0.93, train acc: 0.9290, val f1: 0.83, val acc: 0.8320, time: 260.45s
Epoch:6, Training Loss: 3459.00, Training f1: 0.94, train acc: 0.9383, val f1: 0.83, val acc: 0.8273, time: 260.57s
Epoch:7, Training Loss: 2919.77, Training f1: 0.94, train acc: 0.9437, val f1: 0.84, val acc: 0.8373, time: 264.64s
Epoch:8, Training Loss: 2766.90, Training f1: 0.95, train acc: 0.9543, val f1: 0.83, val acc: 0.8322, time: 264.74s
Epoch:9, Training Loss: 2345.42, Training f1: 0.94, train acc: 0.9448, 

KeyboardInterrupt: ignored